# EMTECH. Proyecto 1
## Introducción a Python

LifeStore es una tienda virtual que maneja una amplia gama de artículos recientemente, la Gerencia de ventas, se percató que la empresa tiene una importante acumulación de inventario. Asimismo, se ha identificado una reducción en las búsquedas de un grupo importante de productos, lo que ha redundado en una disminución sustancial de sus ventas del último trimestre.

In [4]:
# Importamos librerías 
import pandas as pd
import numpy as np
# Importamos archivo de datos 
from lifestore_file import lifestore_products, lifestore_sales, lifestore_searches

In [5]:
# Dataframe de ventas 
matriz_sales = np.array(lifestore_sales)
df_sales = pd.DataFrame(matriz_sales, columns=['id_sale','id_product','score','date','refund']) 
# lifestore_sales = [id_sale, id_product, score (from 1 to 5), date, refund (1 for true or 0 to false)] Refund = reembolso 
df_sales["id_sale"] = pd.to_numeric(df_sales["id_sale"])
df_sales["id_product"] = pd.to_numeric(df_sales["id_product"])
df_sales["score"] = pd.to_numeric(df_sales["score"])
df_sales["refund"] = pd.to_numeric(df_sales["refund"])
df_sales["date"] = pd.to_datetime(df_sales["date"])
df_sales.head()

,id_sale,id_product,score,date,refund
0,1,1,5,2020-07-24,0
1,2,1,5,2020-07-27,0
2,3,2,5,2020-02-24,0
3,4,2,5,2020-05-22,0
4,5,2,5,2020-01-01,0


In [93]:
df_sales.dtypes

id_sale                int64
id_product             int64
score                  int64
date          datetime64[ns]
refund                 int64
dtype: object

In [6]:
# Dataframe de productos
matriz_products = np.array(lifestore_products)
df_products = pd.DataFrame(matriz_products, columns=['id_product','name','price','category','stock'])
df_products["id_product"] = pd.to_numeric(df_products["id_product"])
df_products["price"] = pd.to_numeric(df_products["price"])
df_products["stock"] = pd.to_numeric(df_products["stock"])
df_products.head()

,id_product,name,price,category,stock
0,1,"Procesador AMD Ryzen 3 3300X S-AM4, 3.80GHz, Q...",3019,procesadores,16
1,2,"Procesador AMD Ryzen 5 3600, S-AM4, 3.60GHz, 3...",4209,procesadores,182
2,3,"Procesador AMD Ryzen 5 2600, S-AM4, 3.40GHz, S...",3089,procesadores,987
3,4,Procesador AMD Ryzen 3 3200G con Gráficos Rade...,2209,procesadores,295
4,5,"Procesador Intel Core i3-9100F, S-1151, 3.60GH...",1779,procesadores,130


In [7]:
df_products.dtypes

id_product     int64
name          object
price          int64
category      object
stock          int64
dtype: object

In [8]:
# Dataframe de búsquedas
matriz_searches = np.array(lifestore_searches)
df_searches = pd.DataFrame(matriz_searches, columns=['id_search','id_product'])
# ---- lifestore_searches = [id_search, id product]
df_searches["id_search"] = pd.to_numeric(df_searches["id_search"])
df_searches["id_product"] = pd.to_numeric(df_searches["id_product"])
df_searches.head()

,id_search,id_product
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [9]:
df_searches.dtypes

id_search     int64
id_product    int64
dtype: object

In [16]:
# Ventas
df_ventas = df_sales.groupby(by="id_product").count()
df_ventas = df_ventas.drop(['score','date','refund'], axis=1)
df_ventas = df_ventas.set_axis(['ventas'], axis=1)
df_ventasc = pd.merge(df_products,df_ventas, on='id_product')

In [19]:
# Búsquedas
df_busquedas = df_searches.groupby(by="id_product").count()
df_busquedas = df_busquedas.set_axis(['busquedas'], axis=1)
df_busquedasc = pd.merge(df_products,df_busquedas, on='id_product')

### 1. Productos más vendidos y productos rezagados

In [17]:
# Productos más vendidos 
df_masvendidos = df_ventasc.sort_values(by='ventas', ascending=False).head(5)
df_masvendidos

,id_product,name,price,category,stock,ventas
32,54,"SSD Kingston A400, 120GB, SATA III, 2.5'', 7mm",259,discos duros,300,50
2,3,"Procesador AMD Ryzen 5 2600, S-AM4, 3.40GHz, S...",3089,procesadores,987,42
4,5,"Procesador Intel Core i3-9100F, S-1151, 3.60GH...",1779,procesadores,130,20
22,42,Tarjeta Madre ASRock Micro ATX B450M Steel Leg...,1779,tarjetas madre,0,18
33,57,"SSD Adata Ultimate SU800, 256GB, SATA III, 2.5...",889,discos duros,15,15


In [20]:
# Productos con mayor número de búsquedas 
df_masbuscados = df_busquedasc.sort_values(by='busquedas', ascending=False).head(10)
df_masbuscados

,id_product,name,price,category,stock,busquedas
37,54,"SSD Kingston A400, 120GB, SATA III, 2.5'', 7mm",259,discos duros,300,263
39,57,"SSD Adata Ultimate SU800, 256GB, SATA III, 2.5...",889,discos duros,15,107
22,29,Tarjeta Madre ASUS micro ATX TUF B450M-PLUS GA...,2499,tarjetas madre,10,60
2,3,"Procesador AMD Ryzen 5 2600, S-AM4, 3.40GHz, S...",3089,procesadores,987,55
3,4,Procesador AMD Ryzen 3 3200G con Gráficos Rade...,2209,procesadores,295,41
50,85,"Logitech Audífonos Gamer G635 7.1, Alámbrico, ...",2159,audifonos,39,35
43,67,"TV Monitor LED 24TL520S-PU 24, HD, Widescreen,...",3229,pantallas,411,32
6,7,"Procesador Intel Core i7-9700K, S-1151, 3.60GH...",8559,procesadores,114,31
4,5,"Procesador Intel Core i3-9100F, S-1151, 3.60GH...",1779,procesadores,130,30
31,47,"SSD XPG SX8200 Pro, 256GB, PCI Express, M.2",1209,discos duros,8,30


In [21]:
# Productos con menores ventas
df_menosvendidos = df_ventasc.sort_values(by='ventas', ascending=True).head(5)
df_menosvendidos

,id_product,name,price,category,stock,ventas
41,94,HyperX Audífonos Gamer Cloud Flight para PC/PS...,2869,audifonos,12,1
15,22,Tarjeta de Video MSI NVIDIA GeForce GTX 1050 T...,3429,tarjetas de video,0,1
21,40,"Tarjeta Madre Gigabyte XL-ATX TRX40 Designare,...",17439,tarjetas madre,1,1
24,45,"Tarjeta Madre ASRock ATX H110 Pro BTC+, S-1151...",2869,tarjetas madre,25,1
12,17,Tarjeta de Video Gigabyte AMD Radeon R7 370 OC...,4199,tarjetas de video,1,1


In [22]:
# Productos con menor número de búsquedas 
df_menosbuscados = df_busquedasc.sort_values(by='busquedas', ascending=True).head(10)
df_menosbuscados

,id_product,name,price,category,stock,busquedas
44,70,"Samsung Smart TV LED 43, Full HD, Widescreen, ...",7679,pantallas,10,1
53,93,"Ginga Audífonos con Micrófono GI18ADJ01BT-RO, ...",160,audifonos,139,1
24,35,Tarjeta Madre Gigabyte micro ATX Z390 M GAMING...,3419,tarjetas madre,30,1
48,80,"Ghia Bocina Portátil BX800, Bluetooth, Inalámb...",1359,bocinas,15,1
8,9,"Procesador Intel Core i3-8100, S-1151, 3.60GHz...",2549,procesadores,35,1
9,10,"MSI GeForce 210, 1GB GDDR3, DVI, VGA, HDCP, PC...",889,tarjetas de video,13,1
20,27,"Tarjeta de Video VisionTek AMD Radeon HD5450, ...",2109,tarjetas de video,43,1
40,59,"SSD Samsung 860 EVO, 1TB, SATA III, M.2",5539,discos duros,10,1
29,45,"Tarjeta Madre ASRock ATX H110 Pro BTC+, S-1151...",2869,tarjetas madre,25,1
52,91,"Genius GHP-400S Audífonos, Alámbrico, 1.5 Metr...",137,audifonos,16,2


### 2. Productos por reseña en el servicio


In [73]:
# Reseñas
df_resenas = df_sales.groupby(by=["id_product","score"], as_index=False).count()
df_resenas = df_resenas.set_axis(['id_product','score','resenas','calificacion','promedio'], axis=1)
df_resenas["id_product"] = pd.to_numeric(df_resenas["id_product"])
df_resenas["score"] = pd.to_numeric(df_resenas["score"])
df_resenas["calificacion"] = df_resenas["resenas"]*df_resenas["score"]
df_resenas = df_resenas.groupby(by="id_product").sum()
df_resenas["promedio"] = df_resenas["calificacion"]/df_resenas["resenas"]
df_resenas = pd.merge(df_products,df_resenas, on='id_product')
df_resenas = df_resenas.filter(['id_product','name','promedio'])

In [74]:
# Productos con mejores reseñas 5
df_mejoresscore = df_resenas.sort_values(by='promedio', ascending=False).head(5)
df_mejoresscore

,id_product,name,promedio
0,1,"Procesador AMD Ryzen 3 3300X S-AM4, 3.80GHz, Q...",5.0
14,21,Tarjeta de Video MSI AMD Mech Radeon RX 5500 X...,5.0
39,85,"Logitech Audífonos Gamer G635 7.1, Alámbrico, ...",5.0
38,84,"Logitech Audífonos Gamer G332, Alámbrico, 2 Me...",5.0
36,67,"TV Monitor LED 24TL520S-PU 24, HD, Widescreen,...",5.0


In [75]:
# Productos con peores reseñas
df_peorscore = df_resenas.sort_values(by='promedio', ascending=True).head(5)
df_peorscore

,id_product,name,promedio
24,45,"Tarjeta Madre ASRock ATX H110 Pro BTC+, S-1151...",1.000000
12,17,Tarjeta de Video Gigabyte AMD Radeon R7 370 OC...,1.000000
19,31,Tarjeta Madre AORUS micro ATX B450 AORUS M (re...,1.833333
25,46,"Tarjeta Madre Gigabyte micro ATX GA-H110M-DS2,...",2.000000
40,89,"Cougar Audífonos Gamer Phontum Essential, Alám...",3.000000


### 3. Total de ingresos y ventas promedio mensuales, total anual y meses con más ventas en el año

In [125]:
# Ingresos (quitando devoluciones)
df_sale = df_sales
df_sale['mes'] = df_sale['date'].dt.strftime('%m')
df_sale['anio'] = df_sale['date'].dt.strftime('%Y')
df_sale = df_sale[df_sale['refund'] == 0]
df_sale = pd.merge(df_sale, df_products, on='id_product')

In [127]:
# Total de ingresos anuales
df_ingresos = df_sale.groupby(by=['anio']).sum()
df_ingresos.filter(['price'])

,price
anio,
2020,737916


In [128]:
# Total de ventas anuales
df_ingresos = df_sale.groupby(by=['anio']).count()
df_ingresos.filter(['price'])

,price
anio,
2020,274


In [130]:
# Total de ingresos mensuales
df_ingresos = df_sale.groupby(by=['mes']).sum()
df_ingresos.filter(['price'])

,price
mes,
01,72983
02,107674
03,131649
04,169578
05,69071
06,57314
07,38037
08,7655
09,36916


In [137]:
# Total de ventas mensuales
df_ingresos = df_sale.groupby(by=['mes']).count()
df_ingresos.filter(['price'])

,price
mes,
01,37
02,36
03,41
04,72
05,19
06,16
07,13
08,5
09,14


In [142]:
# Meses con más ventas
df_ingresos = df_sale.groupby(by=['mes']).sum()
df_ingresos = df_ingresos.sort_values(by='price',ascending=False)
df_ingresos.filter(['price'])

,price
mes,
04,169578
03,131649
02,107674
01,72983
05,69071
06,57314
07,38037
09,36916
11,21021


In [3]:
# Autenticación 
usuario = 'yess'
contra = 'yess'
intentos = 0
acceso = False

while not acceso:
  intentos +=1
  if intentos ==4:
    exit()
  if input('Usuario: ') == usuario and input('Contraseña: ') == contra:
    acceso = True
    print('Acceso autorizado')
  else: 
    print(f'Tienes {3 - intentos} intentos restantes')


Usuario: yess
Contraseña: yess
Acceso autorizado
